In [1]:
import os
import tqdm
import os.path as osp
import pandas as pd
import numpy as np
import glob
import spacy
%run env.py
%run src/lib.py
%run src/integration.py
%run src/supervision.py

fix_jupyter_spacy_config()

#corpus_dir = osp.join(DATA_DIR, 'articles', 'corpus', 'corpus_00')
#collection_dir = osp.join(REPO_DATA_DIR, 'brat', 'collection_02')

corpus_dir = osp.join(DATA_DIR, 'articles', 'corpus', 'corpus_01')
collection_dir = osp.join(REPO_DATA_DIR, 'brat', 'collection_02')

corpus_docs_path = osp.join(corpus_dir, 'docs')
corpus_links_dir = osp.join(corpus_dir, 'links')

### Link Docs

Link annotated documents (which should ALL be annotated when present) and unannotated docs in a single directory:

In [2]:
files = []
# Add unannotated text files and THEN annotated text files as links will overwrite
f = glob.glob(osp.join(corpus_docs_path, '*.txt'))
assert f, 'No unannotated text files found'
print('Found {} unnannotated text files'.format(len(f)))
files.extend(f)
f = glob.glob(osp.join(collection_dir, '*.txt'))
assert f, 'No annotated text files found'
print('Found {} annotated text files'.format(len(f)))
files.extend(f)
len(files)

Found 10000 unnannotated text files
Found 89 annotated text files


10089

In [3]:
# Clear existing linked files
for f in glob.glob(osp.join(corpus_links_dir, '*.txt')):
    if osp.islink(f):
        os.unlink(f)
!ls -q $corpus_links_dir/*.txt 2> /dev/null

In [4]:
# Add new links
corpus_files = []
for f in files:
    if not osp.exists(corpus_links_dir):
        os.makedirs(corpus_links_dir)
    path = osp.join(corpus_links_dir, osp.basename(f))
    # Overwrite despite initial clearing as docs from annotation
    # directory are preferred
    if osp.exists(path):
        os.unlink(path)
    os.symlink(f, path)
    corpus_files.append(path)
print('Linked {} files in {}'.format(len(corpus_files), corpus_links_dir))

Linked 10089 files in /lab/data/articles/corpus/corpus_01/links


### Define Pipeline

In [5]:
from spacy.tokens import Doc
from spacy.pipeline import EntityRuler
from spacy.tokens import Span

if not Doc.has_extension('pmc_id'):
    Doc.set_extension('pmc_id', default=None)
if not Span.has_extension('meta'):
    Span.set_extension('meta', default=None)

class EntityFilter(object):
    name = 'filter'
    
    def __init__(self, types):
        self.types = types
        
    def __call__(self, doc):
        if self.types is None:
            return doc
        doc.ents = [e for e in doc.ents if e.label_ in self.types]
        return doc


def copy_ent(ent):
    return dict(
        type=ent.label_,
        meta=ent._.meta if hasattr(ent._, 'meta') else None,
        start_chr=ent.start_char,
        end_chr=ent.end_char,
        start_wrd=ent.start,
        end_wrd=ent.end,
        text=ent.text
    )

class EntityRelocator(object):
    
    def __init__(self, attr):
        self.attr = attr
        if not Doc.has_extension(attr):
            Doc.set_extension(attr, default=None)

    def __call__(self, doc):
        setattr(doc._, self.attr, [copy_ent(e) for e in doc.ents])
        doc.ents = []
        return doc


class EntityRefiner(object):
    name = 'refiner'
    
    def __init__(self, nlp, df):
        self.ruler = EntityRuler(nlp, overwrite_ents=True)
        self.df = df.copy().set_index('id')
        patterns = []
        for rid, r in self.df.iterrows():
            # TODO: add match types into meta data frames
            tokens = nlp.tokenizer(r['sym']) # returns Doc
            pattern = [{'lower': t.lower_} for t in tokens]
                
            # Assign label as record id (e.g. "CKBF6003C60D23BA0D")
            patterns.append({'label': rid, 'pattern': pattern})
        self.ruler.add_patterns(patterns)
        
    def __call__(self, doc):
        doc = self.ruler(doc)
        ents = []
        for ent in doc.ents:
            if ent.label_ not in self.df.index:
                ents.append(ent)
                continue
            # Convert label of span from record id to entity type 
            # (e.g. CKBF6003C60D23BA0D -> CYTOKINE)
            r = self.df.loc[ent.label_]
            rid = ent.label_
            ent = Span(doc, ent.start, ent.end, label=r['type'])
            ent._.meta = dict(id=rid, prefid=r['prefid'], lbl=r['lbl'])
            ents.append(ent)
        doc.ents = ents
        return doc 

In [6]:
def get_pipeline():
    from spacy.tokens import Doc
    from spacy.tokens import Span

    if not Doc.has_extension('pmc_id'):
        Doc.set_extension('pmc_id', default=None)
    if not Span.has_extension('meta'):
        Span.set_extension('meta', default=None)
        
    nlp = get_scispacy_pipeline()
    nlp.remove_pipe('tagger')
    nlp.remove_pipe('parser')
    nlp.add_pipe(EntityFilter(['CELL_TYPE', 'CELL_LINE']), after='ner')
    nlp.add_pipe(EntityRelocator('ents_jnlpba'), after=EntityFilter.name, name='reloc_jnlpba')

    df = pd.concat([
        get_entity_meta_data(CYTOKINES).assign(type=ENT_TYP_CK),
        get_entity_meta_data(TRANSCRIPTION_FACTORS).assign(type=ENT_TYP_TF),
        get_entity_meta_data(CELL_TYPES).assign(type=ENT_TYP_CT)
    ], sort=True)
    nlp.add_pipe(EntityRefiner(nlp, df), after='reloc_jnlpba')
    nlp.add_pipe(EntityRelocator('ents_lkp'), after=EntityRefiner.name, name='reloc_lkp')
    return nlp

nlp = get_pipeline()
nlp.pipeline

[('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f836dddfe88>),
 ('filter', <__main__.EntityFilter at 0x7f836e055320>),
 ('reloc_jnlpba', <__main__.EntityRelocator at 0x7f836e055390>),
 ('refiner', <__main__.EntityRefiner at 0x7f836e055400>),
 ('reloc_lkp', <__main__.EntityRelocator at 0x7f836279f7f0>)]

### Run Pipeline

In [14]:
def process(files):
    nlp = get_pipeline()
    
    def get_text(file):
        doc_id = osp.basename(file).replace('.txt', '')
        with open(file, 'r') as fd:
            text = fd.read()
            # Clip to max length to avoid error like
            # ValueError: [E088] Text of length 1290071 exceeds maximum of 1000000
            if len(text) > nlp.max_length:
                text = text[:nlp.max_length]
            return text, doc_id
            
    def to_df(doc):
        df = []
        ents = [(e, 'jnlpba') for e in doc._.ents_jnlpba] + [(e, 'lkp') for e in doc._.ents_lkp]
        for ent, src in ents:
            meta = ent['meta'] or {}
            df.append({
                **dict(
                    id=doc._.pmc_id, 
                    ent_id=meta.get('id'),
                    ent_lbl=meta.get('lbl'),
                    ent_prefid=meta.get('prefid'),
                    ent_src=src
                ),
                **{k:v for k, v in ent.items() if k != 'meta'}
            })
        return pd.DataFrame(df)
    
    res = []
    for f in tqdm.tqdm(files):
        doc_text, doc_id = get_text(f)
        doc = nlp(doc_text)
        doc._.pmc_id = doc_id
        res.append(to_df(doc))
    return pd.concat(res)

In [ ]:
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=3, processes=True, direct_to_workers=True)
client

In [24]:
import dask
batches = list(np.array_split(files[:1000], 12))
futures = client.map(process, batches)
dask.distributed.progress(futures)

VBox()

In [25]:
pd.set_option('display.max_info_rows', 10000000)
df = pd.concat([r.result() for r in futures])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 161216 entries, 0 to 265
Data columns (total 11 columns):
end_chr       161216 non-null int64
end_wrd       161216 non-null int64
ent_id        81895 non-null object
ent_lbl       81895 non-null object
ent_prefid    81895 non-null object
ent_src       161216 non-null object
id            161216 non-null object
start_chr     161216 non-null int64
start_wrd     161216 non-null int64
text          161216 non-null object
type          161216 non-null object
dtypes: int64(4), object(7)
memory usage: 14.8+ MB


In [ ]:
# Make sure that when the entity source is JNLPBA, all lookup related metadata is null (can happen if spans are mangled)
assert len(df[(df['ent_src'] == 'jnlpba') & (df['ent_id'].notnull())]) == 0
# Also make sure that entity metadata IS present when source is for a direct lookup
assert len(df[(df['ent_src'] == 'lkp') & (df['ent_id'].isnull())]) == 0

In [11]:
# doc = nlp('Some T-helper-17 cells secreting IL2 and STAT3 where CD4 is expressed on surface of antigen-presenting DCs')

In [16]:
# #doc = nlp('The Vγ4 T lymphocyte subset predominated among the IL-17+ cell populations and the TCM in the lungs of CLP mice (unlike Vγ1 and αβ T lymphocytes) and was strongly biased toward IL-17 rather than toward IFN-γ production')
# doc = nlp(u'We found this occurred through the activation of tumor-infiltrating, exhausted CD8+ T cells (CD8+ TILs) that had lost most of the original functions, such as the ability to produce multiple cytokines and cytotoxicity. CD8+ TILs of mice exposed to Met begin to rapidly produce multiple cytokines, including interleukin-2 (IL-2), tumor necrosis factor alpha (TNFα), and interferon gamma (IFNγ), and differentiate into effector memory T cells (TEM); otherwise, central memory T cells (TCM) are dominant in the tumor microenvironment')
# to_df(doc)

In [35]:
# path = '/Users/eczech/data/research/hammer/nlp/20190311-pubmed-tcell-relation/articles/corpus/corpus_01/links/PMC5704053.txt'
# doc = None
# with open(path, 'r') as fd:
#     text = fd.read()
#     doc = nlp(text)
#     #doc._.pmc_id = osp.basename(path).replace('.txt', '')

### Tag Export

In [16]:
pd.set_option('display.max_info_rows', 10000000)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2310512 entries, 0 to 78
Data columns (total 11 columns):
end_chr       2310512 non-null int64
end_wrd       2310512 non-null int64
ent_id        961277 non-null object
ent_lbl       961277 non-null object
ent_prefid    961277 non-null object
ent_src       2310512 non-null object
id            2310512 non-null object
start_chr     2310512 non-null int64
start_wrd     2310512 non-null int64
text          2310512 non-null object
type          2310512 non-null object
dtypes: int64(4), object(7)
memory usage: 211.5+ MB


In [21]:
tags_file = osp.join(corpus_dir, 'tags-union.csv')
df.to_csv(tags_file, index=False)
tags_file

'/lab/data/articles/corpus/corpus_01/tags-union.csv'

### Relation Export

Export relations to be used to generate gold labellings within Snorkel

In [18]:
from bratreader.repomodel import RepoModel
repo_model = RepoModel(collection_dir)

In [19]:
len(repo_model.documents)

89

In [24]:
# repo_model.documents.keys()

In [25]:
# doc = repo_model.documents['PMC2646571']
# annot = doc.annotations[1]

In [26]:
# annot.labels, annot.links, annot.realspan, annot.spans, annot.repr, annot.words

In [27]:
# doc.key

In [20]:
def get_relations(doc):
    relations = []
    
    # Map from entity type label used in annotation to label used within snorkel
    ent_typ_map = {
        'TF': ENT_TYP_TF,
        'CELL_TYPE': ENT_TYP_CT,
        'CYTOKINE': ENT_TYP_CK
    }
    
    for a2 in doc.annotations:
        # Links only exist as INCOMING links meaning that a2 is the target of relations
        # and a1 will be the source
        if a2.links:
            for relation_type, annots in a2.links.items():
                for a1 in annots:
                    assert len(a1.labels) == len(a2.labels) == 1
                    a2_typ = ent_typ_map[list(a2.labels.keys())[0]]
                    
                    # For consistency with snorkel convention, ensure that the second
                    # value in relation entry is the cell type and all others are first
                    e1, e2 = a1, a2
                    if a2_typ != ENT_TYP_CT:
                        e1, e2 = a2, a1
                    
                    relations.append(dict(
                        id=doc.key,
                        rel_typ=relation_type, 
                        e1_typ=ent_typ_map[list(e1.labels.keys())[0]], 
                        e1_start_chr=e1.realspan[0], 
                        e1_end_chr=e1.realspan[1], 
                        e1_text=e1.repr,
                        e2_typ=ent_typ_map[list(e2.labels.keys())[0]], 
                        e2_start_chr=e2.realspan[0], 
                        e2_end_chr=e2.realspan[1], 
                        e2_text=e2.repr
                    ))
    return relations
df = pd.DataFrame([r for k, doc in repo_model.documents.items() for r in get_relations(doc)])
df = df.drop_duplicates()
df.head()

,e1_end_chr,e1_start_chr,e1_text,e1_typ,e2_end_chr,e2_start_chr,e2_text,e2_typ,id,rel_typ
0,24,19,Gfi-1,TRANSCRIPTION_FACTOR,85,81,Th17,IMMUNE_CELL_TYPE,PMC2646571,Differentiation
1,44,39,TGF-β,CYTOKINE,85,81,Th17,IMMUNE_CELL_TYPE,PMC2646571,Induction
2,44,39,TGF-β,CYTOKINE,119,97,inducible regulatory T,IMMUNE_CELL_TYPE,PMC2646571,Induction
3,24,19,Gfi-1,TRANSCRIPTION_FACTOR,119,97,inducible regulatory T,IMMUNE_CELL_TYPE,PMC2646571,Differentiation
8,371,366,Gfi-1,TRANSCRIPTION_FACTOR,436,433,Th2,IMMUNE_CELL_TYPE,PMC2646571,Differentiation


In [21]:
df.groupby(['e1_typ', 'e2_typ', 'rel_typ']).size()

e1_typ                e2_typ            rel_typ        
CYTOKINE              IMMUNE_CELL_TYPE  Induction          150
                                        Secretion          131
TRANSCRIPTION_FACTOR  IMMUNE_CELL_TYPE  Differentiation    119
dtype: int64

In [22]:
#df_exp[df_exp['id'] == 'PMC3304099']

# Export

In [23]:
path = osp.join(corpus_dir, 'relations.csv')
df.to_csv(path, index=False)
path

'/Users/eczech/data/research/hammer/nlp/20190311-pubmed-tcell-relation/articles/corpus/corpus_01/relations.csv'

<hr>

### Tag EDA

In [22]:
import pandas as pd
import os.path as osp
df = pd.read_csv(osp.join(corpus_dir, 'tags.csv'))
df.head()

,end_chr,end_wrd,ent_id,ent_lbl,ent_prefid,ent_src,id,start_chr,start_wrd,text,type
0,828,148,NaN,NaN,NaN,jnlpba,PMC100031,821,146,B cells,CELL_TYPE
1,957,170,NaN,NaN,NaN,jnlpba,PMC100031,912,165,immunoglobulin M (IgM)-secreting spleen cells,CELL_TYPE
2,1012,181,NaN,NaN,NaN,jnlpba,PMC100031,991,178,IgM-secreting B cells,CELL_TYPE
3,1375,239,NaN,NaN,NaN,jnlpba,PMC100031,1340,236,non-parasite-directed B-cell clones,CELL_LINE
4,1697,292,NaN,NaN,NaN,jnlpba,PMC100031,1684,288,B and T cells,CELL_TYPE


In [23]:
df['type'].value_counts()

CELL_TYPE               1054510
CYTOKINE                 485040
IMMUNE_CELL_TYPE         314554
CELL_LINE                294725
TRANSCRIPTION_FACTOR     161683
Name: type, dtype: int64

In [24]:
(
    df
    .pipe(lambda df: df[df['type'] == ENT_TYP_TF])
    .groupby('ent_lbl').size()
    .sort_values(ascending=False)
    .head(35)
)

ent_lbl
FOXP3     23964
STAT3      9437
TBX21      9248
RORC       7017
GATA3      4697
STAT1      3398
BCL6       2709
PRDM6      2656
IRF4       2504
EOMES      2390
PRDM1      2241
SPI1       2016
STAT6      1846
FOXO1      1824
NFATC1     1774
STAT5B     1772
CHAMP1     1760
STAT5A     1746
RUNX1      1739
IRF8       1721
STAT4      1604
MYC        1324
NR2E3      1319
BCL11B     1238
BATF       1170
JUNB       1151
MAF        1150
NFATC2     1097
POU5F1      905
JUN         888
CREB1       855
RUNX3       848
TCF7        831
ZBTB16      826
KLF4        810
dtype: int64

In [25]:
(
    df
    .pipe(lambda df: df[df['type'] == ENT_TYP_TF])
    .pipe(lambda df: df[df['ent_lbl'] == 'RELA'])
    .assign(text=lambda df: df['text'].str.upper())
    .groupby(['id', 'ent_lbl', 'text'])
    .size().sort_values(ascending=False)   
    .head(15)
)

id          ent_lbl  text
PMC4734616  RELA     RELA    41
PMC2196335  RELA     RELA    25
PMC4920892  RELA     RELA    22
PMC4821646  RELA     RELA    18
PMC5152706  RELA     RELA    15
PMC3543335  RELA     RELA    15
PMC6087740  RELA     RELA    14
PMC5924754  RELA     RELA    14
PMC4172221  RELA     RELA    12
PMC3724937  RELA     RELA    11
PMC3823077  RELA     RELA    10
PMC4695108  RELA     RELA    10
PMC6435687  RELA     RELA     8
PMC4846868  RELA     RELA     7
PMC4491573  RELA     RELA     7
dtype: int64

In [21]:
#!cat $corpus_dir/docs/PMC4734616.txt